In [12]:
# Install dependencies
%pip install pennylane
%pip install qiskit qiskit_machine_learning
%pip install numpy pandas matplotlib scipy scikit-learn
%pip install folium
%pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Project - Quantum Approach to Biodiversity Mapping & Predictions

Figma [here](https://www.figma.com/board/YJkl666NgYY9lzeGnKZ1lw/Quantum-Hackathon-2024?node-id=0-1&node-type=canvas&t=7ul1ZMUwhKcWZdU6-0)

Goals:  
1.  
2.  

In [29]:
# Import libraries
import numpy as np
import pandas as pd
import pennylane as qml
import qiskit
import zipfile
import io
import os
import folium
import ipywidgets as widgets
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes
from qiskit_machine_learning.algorithms import VQR
from qiskit_machine_learning.circuit.library import RawFeatureVector
from folium.plugins import TimestampedGeoJson, MarkerCluster
from IPython.display import display
from ipywidgets import interact, IntSlider, Select, Layout


In [32]:
weather_df = pd.read_parquet('daily_weather.parquet', engine='fastparquet')
weather_df.head()


,station_id,city_name,date,season,avg_temp_c,min_temp_c,max_temp_c,precipitation_mm,snow_depth_mm,avg_wind_dir_deg,avg_wind_speed_kmh,peak_wind_gust_kmh,avg_sea_level_pres_hpa,sunshine_total_min
0,41515,Asadabad,1957-07-01,Summer,27.0,21.1,35.6,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,41515,Asadabad,1957-07-02,Summer,22.8,18.9,32.2,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,41515,Asadabad,1957-07-03,Summer,24.3,16.7,35.6,1.0,NaN,NaN,NaN,NaN,NaN,NaN
3,41515,Asadabad,1957-07-04,Summer,26.6,16.1,37.8,4.1,NaN,NaN,NaN,NaN,NaN,NaN
4,41515,Asadabad,1957-07-05,Summer,30.8,20.0,41.7,0.0,NaN,NaN,NaN,NaN,NaN,NaN


# Data

[BioTIME database](https://zenodo.org/records/5026943#.Y9ZAKdJBwUE)

Data Citation:
Dornelas M, Antão LH, Moyes F, Bates, AE, Magurran, AE, et al. BioTIME: A database of biodiversity time series for the Anthropocene. Global Ecol Biogeogr. 2018; 27:760 - 786. https://doi.org/10.1111/geb.12729

## Load Data

metadata and biodiversity data
see [here](https://towardsdatascience.com/different-ways-to-connect-google-drive-to-a-google-colab-notebook-pt-1-de03433d2f7a)
effectively need to upload the data to gdrive as collab storage is transient

In [14]:
# Load metadata
metadata_df  = pd.read_csv("BioTIMEMetadata_24_06_2021.csv", encoding='latin1')
# Strip any leading/trailing spaces from column names
metadata_df.columns = metadata_df.columns.str.strip()
# Display the first few rows of the metadata
metadata_df.head()

,STUDY_ID,REALM,CLIMATE,GENERAL_TREAT,TREATMENT,TREAT_COMMENTS,TREAT_DATE,HABITAT,PROTECTED_AREA,BIOME_MAP,...,WEB_LINK,DATA_SOURCE,METHODS,SUMMARY_METHODS,LINK_ID,COMMENTS,DATE_STUDY_ADDED,ABUNDANCE_TYPE,BIOMASS_TYPE,SAMPLE_DESC_NAME
0,10,Terrestrial,Temperate,NaN,NaN,NaN,NaN,Woodland,False,Temperate broadleaf and mixed forests,...,http://esapubs.org/archive/ecol/E082/011/defau...,Ecology,Itasca State Park. Minnesota. is a 13 000-ha f...,Plots,NaN,Dates added,Oct-12,Count,NaN,lat_long_treefallid_controlvs.treatment_basevs...
1,18,Terrestrial,Temperate,NaN,NaN,NaN,NaN,Sagebrush steppe,False,Deserts and xeric shrublands,...,http://esapubs.org/archive/ecol/E091/243/defau...,Ecology,Site descriptionThe U.S. Sheep Experiment Stat...,Plots,NaN,Dates added Years (1923. 1926. 1927. 1929) del...,Oct-12,Count,NaN,lat_long_quadrat_year
2,33,Marine,Temperate,NaN,NaN,NaN,NaN,Seaweed beds,False,Temperate shelf and seas ecoregions,...,http://plankt.oxfordjournals.org/content/32/5.toc,Oxford Journals,Sampling and enumeration of phytoplanktonWeekl...,Stations,NaN,Inaccurate decimal latitude/longitude conversi...,Oct-12,Count,NaN,lat_long_location_method_date
3,39,Terrestrial,Temperate,NaN,NaN,NaN,NaN,Deciduous forest,False,Temperate broadleaf and mixed forests,...,http://www.esajournals.org/toc/emon/56/3,Ecology,This study was conducted in the Hubbard Brook ...,Plots,NaN,Removed records prior to 1986 as per provider ...,Oct-12,Density,NaN,lat_long_timeTransect_year
4,41,Terrestrial,Temperate,NaN,NaN,NaN,NaN,Woodland,False,Temperate broadleaf and mixed forests,...,http://www.esajournals.org/toc/ecol/41/4,Ecology,Time and space and the variation of species - ...,Counts,NaN,Dates added,Oct-12,Count,NaN,lat_long_census_year


In [15]:
# load and extract data file 

with zipfile.ZipFile("BioTIMEQuery_24_06_2021.zip") as z:
    print(z.namelist())
    with z.open("BioTIMEQuery_24_06_2021.csv") as f:
        data_df = pd.read_csv(f, encoding='latin1')
# data_df = pd.read_csv("BioTIMEQuery_24_06_2021.csv", encoding='latin1')
# show the first few rows of the data
data_df.head()

['BioTIMEQuery_24_06_2021.csv']


/var/folders/_r/b4tg4wss2j99nxws_4xy7mk80000gn/T/ipykernel_98805/2856869001.py:6: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  data_df = pd.read_csv(f, encoding='latin1')


,Unnamed: 0,STUDY_ID,DAY,MONTH,YEAR,SAMPLE_DESC,PLOT,ID_SPECIES,LATITUDE,LONGITUDE,sum.allrawdata.ABUNDANCE,sum.allrawdata.BIOMASS,GENUS,SPECIES,GENUS_SPECIES
0,1,10,NaN,NaN,1984,47.400000_-95.120000_12_Control_0_Medium,12,22,47.4,-95.12,1.0,0.0,Acer,rubrum,Acer rubrum
1,2,10,NaN,NaN,1984,47.400000_-95.120000_12_Control_0_Medium,12,23,47.4,-95.12,3.0,0.0,Acer,saccharum,Acer saccharum
2,3,10,NaN,NaN,1984,47.400000_-95.120000_12_Control_0_Medium,12,24,47.4,-95.12,1.0,0.0,Acer,spicatum,Acer spicatum
3,4,10,NaN,NaN,1984,47.400000_-95.120000_12_Control_0_Medium,12,607,47.4,-95.12,12.0,0.0,Corylus,cornuta,Corylus cornuta
4,5,10,NaN,NaN,1984,47.400000_-95.120000_12_Control_0_Small,12,1911,47.4,-95.12,1.0,0.0,Populus,pinnata,Populus pinnata


In [16]:
# show the last few rows of the data
data_df.tail()

,Unnamed: 0,STUDY_ID,DAY,MONTH,YEAR,SAMPLE_DESC,PLOT,ID_SPECIES,LATITUDE,LONGITUDE,sum.allrawdata.ABUNDANCE,sum.allrawdata.BIOMASS,GENUS,SPECIES,GENUS_SPECIES
8552244,26178100,548,NaN,NaN,2007,49.1014548954342_13.3200349605548_T3_56_2007,T3_56,49340,49.10146,13.32004,3.0,NaN,Vaccinium,vitis.idaea,Vaccinium vitis.idaea
8552245,26179100,548,NaN,NaN,2009,49.1014548954342_13.3200349605548_T3_56_2009,T3_56,49340,49.10146,13.32004,4.0,NaN,Vaccinium,vitis.idaea,Vaccinium vitis.idaea
8552246,26180100,548,NaN,NaN,2012,49.1014548954342_13.3200349605548_T3_56_2012,T3_56,49340,49.10146,13.32004,3.0,NaN,Vaccinium,vitis.idaea,Vaccinium vitis.idaea
8552247,26181100,548,NaN,NaN,2007,49.097317976565_13.3173542074378_T3_51_2007,T3_51,40355,49.09732,13.31735,10.0,NaN,Veronica,chamaedrys,Veronica chamaedrys
8552248,26182100,548,NaN,NaN,2011,49.1014548954342_13.3200349605548_T3_56_2011,T3_56,40355,49.10146,13.32004,4.0,NaN,Veronica,chamaedrys,Veronica chamaedrys


In [17]:
# Strip any leading/trailing spaces from column names
data_df.columns = data_df.columns.str.strip()
data_df.describe()

,Unnamed: 0,STUDY_ID,DAY,MONTH,YEAR,ID_SPECIES,LATITUDE,LONGITUDE,sum.allrawdata.ABUNDANCE,sum.allrawdata.BIOMASS
count,8.552249e+06,8.552249e+06,7.108710e+06,7.132111e+06,8.552249e+06,8.552249e+06,8.552249e+06,8.552249e+06,8.552126e+06,8.525108e+06
mean,4.335884e+06,2.245117e+02,1.442245e+01,6.520774e+00,1.993227e+03,1.767708e+04,2.836394e+01,-4.049484e+01,1.095608e+04,6.540004e+06
std,2.959943e+06,1.051472e+02,9.120689e+00,2.808826e+00,1.522319e+01,1.586367e+04,3.131463e+01,9.091774e+01,2.312769e+06,9.405228e+09
min,1.000000e+00,1.000000e+01,1.000000e+00,1.000000e+00,1.874000e+03,1.000000e+00,-8.898167e+01,-1.800000e+02,0.000000e+00,0.000000e+00
25%,2.138366e+06,1.480000e+02,6.000000e+00,5.000000e+00,1.987000e+03,4.975000e+03,2.731000e+01,-9.658333e+01,1.000000e+00,0.000000e+00
50%,4.276428e+06,1.950000e+02,1.400000e+01,7.000000e+00,1.996000e+03,8.606000e+03,3.908333e+01,-7.170000e+01,1.000000e+00,0.000000e+00
75%,6.424869e+06,3.020000e+02,2.200000e+01,8.000000e+00,2.003000e+03,3.668300e+04,4.603832e+01,-3.687600e+00,6.000000e+00,6.000000e-01
max,8.546100e+07,5.480000e+02,3.100000e+01,1.200000e+01,2.018000e+03,5.206400e+04,8.970000e+01,1.800000e+02,6.400000e+09,2.723760e+13


In [18]:
# Filter the dataframe to include only the latest 10 years
latest_year = data_df['YEAR'].max()
filtered_data_df = data_df[data_df['YEAR'] >= (latest_year - 9)]

# Drop the DAY and MONTH columns if they exist
columns_to_drop = ['DAY', 'MONTH']
filtered_data_df = filtered_data_df.drop(columns=[col for col in columns_to_drop if col in filtered_data_df.columns])

# Display the filtered dataframe
filtered_data_df.head()


,Unnamed: 0,STUDY_ID,YEAR,SAMPLE_DESC,PLOT,ID_SPECIES,LATITUDE,LONGITUDE,sum.allrawdata.ABUNDANCE,sum.allrawdata.BIOMASS,GENUS,SPECIES,GENUS_SPECIES
10138,10139,33,2009,50.25_-4.217_L4_Station_English_Channel_10L_sa...,NaN,1158,50.25,-4.217,80.0,0.0,Guinardia,delicatula,Guinardia delicatula
10139,10140,33,2009,50.25_-4.217_L4_Station_English_Channel_10L_sa...,NaN,1189,50.25,-4.217,80.0,0.0,Haslea,wawrikae,Haslea wawrikae
10140,10141,33,2009,50.25_-4.217_L4_Station_English_Channel_10L_sa...,NaN,1648,50.25,-4.217,400.0,0.0,Navicula,spec,Navicula spec
10141,10142,33,2009,50.25_-4.217_L4_Station_English_Channel_10L_sa...,NaN,1677,50.25,-4.217,1520.0,0.0,Nitzschia,closterium,Nitzschia closterium
10142,10143,33,2009,50.25_-4.217_L4_Station_English_Channel_10L_sa...,NaN,1680,50.25,-4.217,320.0,0.0,Nitzschia,sigmoidea,Nitzschia sigmoidea


In [19]:
# Display the filtered dataframe
filtered_data_df.tail()

,Unnamed: 0,STUDY_ID,YEAR,SAMPLE_DESC,PLOT,ID_SPECIES,LATITUDE,LONGITUDE,sum.allrawdata.ABUNDANCE,sum.allrawdata.BIOMASS,GENUS,SPECIES,GENUS_SPECIES
8552242,26176100,548,2010,49.0981453488146_13.3178902393319_T3_52_2010,T3_52,49340,49.09815,13.31789,1.0,NaN,Vaccinium,vitis.idaea,Vaccinium vitis.idaea
8552243,26177100,548,2012,49.0981453488146_13.3178902393319_T3_52_2012,T3_52,49340,49.09815,13.31789,0.5,NaN,Vaccinium,vitis.idaea,Vaccinium vitis.idaea
8552245,26179100,548,2009,49.1014548954342_13.3200349605548_T3_56_2009,T3_56,49340,49.10146,13.32004,4.0,NaN,Vaccinium,vitis.idaea,Vaccinium vitis.idaea
8552246,26180100,548,2012,49.1014548954342_13.3200349605548_T3_56_2012,T3_56,49340,49.10146,13.32004,3.0,NaN,Vaccinium,vitis.idaea,Vaccinium vitis.idaea
8552248,26182100,548,2011,49.1014548954342_13.3200349605548_T3_56_2011,T3_56,40355,49.10146,13.32004,4.0,NaN,Veronica,chamaedrys,Veronica chamaedrys


# Visualisation

In [20]:
# gather useful aggregates
default_lat = filtered_data_df['LATITUDE'].mean()
default_lon = filtered_data_df['LONGITUDE'].mean()

In [21]:
def update_map(year, species_filter='All Species'):

    filtered_data = filtered_data_df[filtered_data_df['YEAR'] == year].copy()
    
    # species filter if not "All Species"
    if species_filter != 'All Species':
        filtered_data = filtered_data[filtered_data['GENUS_SPECIES'] == species_filter]
    
    # Use the mean of valid coordinates, if no valid data points -> use default center
    if len(filtered_data) == 0:
        map_center = [default_lat, default_lon]
    else:
        map_center = [filtered_data['LATITUDE'].mean(), filtered_data['LONGITUDE'].mean()]

    m = folium.Map(location=map_center, zoom_start=4)
    
    # add markers with data
    marker_cluster = MarkerCluster().add_to(m)
    for idx, row in filtered_data.iterrows():
        # for debugging for now, can make it look nicer later
        popup_content = f"""
            <b>Species:</b> {row['GENUS_SPECIES']}<br>
            <b>Abundance:</b> {row['sum.allrawdata.ABUNDANCE']}<br>
            <b>Biomass:</b> {row['sum.allrawdata.BIOMASS']}<br>
            <b>Plot:</b> {row['PLOT']}<br>
            <b>Location:</b> ({row['LATITUDE']}, {row['LONGITUDE']})
        """
        folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            popup=folium.Popup(popup_content, max_width=300),
            tooltip=row['GENUS_SPECIES']
        ).add_to(marker_cluster)
    
    title_html = f'''
        <div style="position: fixed; 
                    top: 10px; 
                    left: 50px; 
                    width: 300px; 
                    height: 30px; 
                    z-index:9999; 
                    background-color: white; 
                    font-size:16px;
                    font-weight: bold;
                    padding: 5px;
                    border-radius: 5px;
                    border: 2px solid gray;">
                Species Distribution Map {year} ({len(filtered_data)} locations)
        </div>
    '''
    m.get_root().html.add_child(folium.Element(title_html))
    return m
        

In [22]:


def create_map(data_df):

    year_slider = IntSlider(
        min=int(data_df['YEAR'].min()),
        max=int(data_df['YEAR'].max()),
        step=1,
        description='Year',
        value=int(data_df['YEAR'].min()),
        layout=Layout(width='800px') 
    )
    
    # species filter - ignore rows that have no valid lat/long
    valid_data = data_df.dropna(subset=['LATITUDE', 'LONGITUDE'])
    species_list = ['All Species'] + sorted(valid_data['GENUS_SPECIES'].unique().tolist())
    species_dropdown = Select(
        options=species_list,
        description='Species:',
        value='All Species'
    )
    
    return interact(update_map, year=year_slider, species_filter=species_dropdown)

In [23]:
create_map(filtered_data_df)

interactive(children=(IntSlider(value=2009, description='Year', layout=Layout(width='800px'), max=2018, min=20…

<function __main__.update_map(year, species_filter='All Species')>

## Quantum Processing

In [24]:
n_qubits = 4
n_layers = 2
scaler = MinMaxScaler()

In [26]:
def prepare_data(data_df):
    # scaling/select relevant features
    features = ['LATITUDE', 'LONGITUDE', 'YEAR','sum.allrawdata.ABUNDANCE']
    
    # clean
    processed_data = data_df[features].dropna()
    
    base_year = processed_data['YEAR'].min()
    processed_data['YEARS_SINCE_START'] = processed_data['YEAR'] - base_year
    
    # normalise
    X = processed_data[['LATITUDE', 'LONGITUDE', 'YEARS_SINCE_START']]
    y = processed_data['sum.allrawdata.ABUNDANCE']
    
    X_scaled = scaler.fit_transform(X)
    y_scaled = MinMaxScaler().fit_transform(y.values.reshape(-1, 1))
    
    return X_scaled, y_scaled

In [27]:
def create_pennylane_circuit():
    # a PennyLane quantum circuit (variational model) for classification
    # see https://pennylane.ai/qml/demos/tutorial_variational_classifier/

    dev = qml.device("default.qubit", wires=n_qubits)
    
    @qml.qnode(dev)
    def circuit(inputs, weights):
        # encode inputs
        for i in range(n_qubits):
            qml.RY(inputs[i % len(inputs)], wires=i)
        
        # variational layers
        for layer in range(n_layers):
            # entangling layer
            for i in range(n_qubits):
                qml.RZ(weights[layer, i], wires=i)
                
            for i in range(n_qubits - 1):
                qml.CNOT(wires=[i, i + 1])
            
            # rotation layer
            for i in range(n_qubits):
                qml.RY(weights[layer + n_layers, i], wires=i)
        
        return qml.expval(qml.PauliZ(0))
    
    return circuit

In [28]:
def train_pennylane_model(X_train, y_train, n_epochs=100):

    # train the model
    circuit = create_pennylane_circuit()

    weights = np.random.uniform(
        low=-np.pi, 
        high=np.pi, 
        size=(2 * n_layers, n_qubits)
    )

    # optimiser https://docs.pennylane.ai/en/stable/code/api/pennylane.GradientDescentOptimizer.html
    opt = qml.GradientDescentOptimizer(stepsize=0.01)

    # training loop
    losses = []
    for epoch in range(n_epochs):
        batch_loss = 0
        for X_batch, y_batch in zip(X_train, y_train):
            # forwards
            prediction = circuit(X_batch, weights)
            loss = np.abs(prediction - y_batch[0])
            
            # back
            weights = opt.step(lambda w: circuit(X_batch, w), weights)
            
            batch_loss += loss
            
        avg_loss = batch_loss / len(X_train)
        losses.append(avg_loss)
        
        if epoch % 10 == 0:
            print(f"Epoch {epoch}: Loss = {avg_loss:.4f}")
            
    return weights, losses

In [22]:
# if we wanna use qiskit instead/penny lane doesn't work
def train_qiskit_model(X_train, y_train, n_epochs=100):

    # https://qiskit-community.github.io/qiskit-machine-learning/stubs/qiskit_machine_learning.algorithms.VQR.html
    # or something else? this seems suitable if penny lane doesnt work
   

SyntaxError: incomplete input (1984441808.py, line 6)

In [29]:
def predict_future_abundance(trained_model, location, years_ahead):
        
    future_predictions = []
    base_input = np.array([
        location[0],  # lat
        location[1],  # long
        scaler.transform([[years_ahead]])[0][0]  # scaled future year
    ])
    
    prediction = trained_model.predict([base_input])[0]
    return prediction

In [30]:
X_scaled, y_scaled = prepare_data(filtered_data_df)

# split into test/train subsets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=123)

# train
penny_lane_model = train_pennylane_model(X_train, y_train)

# prediction
location = [47.4, -95.12] # random lat /long - need to plot these probably
years_ahead = 5
prediction = predict_future_abundance(penny_lane_model, location, years_ahead)

print(prediction)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pennylane/_grad.py:216: UserWarning: Attempted to differentiate a function with no trainable parameters. If this is unintended, please add trainable parameters via the 'requires_grad' attribute or 'argnum' keyword.
  warnings.warn(


KeyboardInterrupt: 